In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [3]:
train.head(10)

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,ID_-0.510_29.290_2019_00,-0.51,29.29,2019,0,-0.000108,0.603019,-0.000065,0.255668,-98.593887,...,3664.436218,61085.809570,2615.120483,15.568533,0.272292,-12.628986,35.632416,-138.786423,30.752140,3.750994
1,ID_-0.510_29.290_2019_01,-0.51,29.29,2019,1,0.000021,0.728214,0.000014,0.130988,16.592861,...,3651.190311,66969.478735,3174.572424,8.690601,0.256830,30.359375,39.557633,-145.183930,27.251779,4.025176
2,ID_-0.510_29.290_2019_02,-0.51,29.29,2019,2,0.000514,0.748199,0.000385,0.110018,72.795837,...,4216.986492,60068.894448,3516.282669,21.103410,0.251101,15.377883,30.401823,-142.519545,26.193296,4.231381
3,ID_-0.510_29.290_2019_03,-0.51,29.29,2019,3,NaN,NaN,NaN,NaN,NaN,...,5228.507736,51064.547339,4180.973322,15.386899,0.262043,-11.293399,24.380357,-132.665828,28.829155,4.305286
4,ID_-0.510_29.290_2019_04,-0.51,29.29,2019,4,-0.000079,0.676296,-0.000048,0.121164,4.121269,...,3980.598120,63751.125781,3355.710107,8.114694,0.235847,38.532263,37.392979,-141.509805,22.204612,4.347317
5,ID_-0.510_29.290_2019_05,-0.51,29.29,2019,5,0.000294,0.871713,0.000242,0.227656,-13.453690,...,5530.354546,49846.001010,4495.301362,7.556143,0.250228,30.128401,37.007020,-137.388719,21.157820,4.310819
6,ID_-0.510_29.290_2019_06,-0.51,29.29,2019,6,-0.000285,0.791956,-0.000226,0.119397,72.198647,...,4378.614648,58804.276563,3537.050659,10.493107,0.240828,4.585368,30.491541,-128.196338,21.721202,4.269334
7,ID_-0.510_29.290_2019_07,-0.51,29.29,2019,7,-0.000016,0.976311,-0.000016,0.106799,-100.956055,...,3468.924146,63697.007267,2597.071563,20.533600,0.250168,16.374829,35.840248,-122.250699,20.432005,4.251361
8,ID_-0.510_29.290_2019_08,-0.51,29.29,2019,8,-0.000115,0.796941,-0.000101,0.164971,-40.179074,...,3160.264670,67222.556159,2530.851476,24.253928,0.260543,-12.021751,42.688789,-114.296369,22.365602,4.281937
9,ID_-0.510_29.290_2019_09,-0.51,29.29,2019,9,0.000056,0.998541,0.000014,0.157726,-100.108744,...,4141.073340,60298.717969,3510.293652,13.813379,0.289295,-66.258392,28.596211,-105.687177,21.084021,4.352933


In [4]:
duplicate_rows_data = train[train.duplicated()]
print("Number of duplicate rows: ", duplicate_rows_data.shape)

Number of duplicate rows:  (0, 76)


In [5]:
desc = pd.DataFrame(index = list(train))
desc['count'] = train.count()
desc['nunique'] = train.nunique()
desc['%unique'] = desc['nunique'] / len(train) * 100
desc['null'] = train.isnull().sum()
desc['type'] = train.dtypes
desc = pd.concat([desc, train.describe().T.drop('count', axis = 1)], axis = 1)
desc

,count,nunique,%unique,null,type,mean,std,min,25%,50%,75%,max
ID_LAT_LON_YEAR_WEEK,79023,79023,100.000000,0,object,NaN,NaN,NaN,NaN,NaN,NaN,NaN
latitude,79023,452,0.571985,0,float64,-1.891072,0.694522,-3.299000,-2.451000,-1.882000,-1.303000,-0.510000
longitude,79023,453,0.573251,0,float64,29.880155,0.810375,28.228000,29.262000,29.883000,30.471000,31.532000
year,79023,3,0.003796,0,int64,2020.000000,0.816502,2019.000000,2019.000000,2020.000000,2021.000000,2021.000000
week_no,79023,53,0.067069,0,int64,26.000000,15.297155,0.000000,13.000000,26.000000,39.000000,52.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
Cloud_sensor_azimuth_angle,78539,78538,99.386255,484,float64,-10.784832,30.374462,-102.739731,-30.309170,-12.673914,9.402202,78.223037
Cloud_sensor_zenith_angle,78539,78539,99.387520,484,float64,40.436976,6.428216,2.998873,35.829907,41.119630,44.446272,65.951248
Cloud_solar_azimuth_angle,78539,78539,99.387520,484,float64,-86.800583,37.837269,-153.464211,-125.991158,-84.644352,-48.132701,-22.653170
Cloud_solar_zenith_angle,78539,78539,99.387520,484,float64,27.925981,4.403835,10.818288,24.686763,28.333630,31.499883,42.060436


In [6]:
train.dtypes

ID_LAT_LON_YEAR_WEEK           object
latitude                      float64
longitude                     float64
year                            int64
week_no                         int64
                               ...   
Cloud_sensor_azimuth_angle    float64
Cloud_sensor_zenith_angle     float64
Cloud_solar_azimuth_angle     float64
Cloud_solar_zenith_angle      float64
emission                      float64
Length: 76, dtype: object

In [7]:
def feat_engineering(X):
    X = X.fillna(method='ffill')
    
    return X

In [8]:
y = train[['emission']]
train = train[["latitude", "longitude", "year", "week_no"]]
X = feat_engineering(train)

ids = test['ID_LAT_LON_YEAR_WEEK']
test = test[["latitude", "longitude", "year", "week_no"]]
test = feat_engineering(test)

In [9]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79023 entries, 0 to 79022
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   79023 non-null  float64
 1   longitude  79023 non-null  float64
 2   year       79023 non-null  int64  
 3   week_no    79023 non-null  int64  
dtypes: float64(2), int64(2)
memory usage: 2.4 MB


In [10]:
X.head(10)

,latitude,longitude,year,week_no
0,-0.51,29.29,2019,0
1,-0.51,29.29,2019,1
2,-0.51,29.29,2019,2
3,-0.51,29.29,2019,3
4,-0.51,29.29,2019,4
5,-0.51,29.29,2019,5
6,-0.51,29.29,2019,6
7,-0.51,29.29,2019,7
8,-0.51,29.29,2019,8
9,-0.51,29.29,2019,9


In [11]:
y.head(10)

,emission
0,3.750994
1,4.025176
2,4.231381
3,4.305286
4,4.347317
5,4.310819
6,4.269334
7,4.251361
8,4.281937
9,4.352933


In [12]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(min_samples_leaf=8, random_state=42).fit(X, y)

c:\users\jason nguyen\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
test_pred = model.predict(test)

submit = pd.DataFrame()
submit['ID_LAT_LON_YEAR_WEEK'] = ids
submit['emission'] = test_pred

submit.to_csv('submission.csv', index=False)

In [14]:
submit.head(20)

,ID_LAT_LON_YEAR_WEEK,emission
0,ID_-0.510_29.290_2022_00,3.993243
1,ID_-0.510_29.290_2022_01,3.993243
2,ID_-0.510_29.290_2022_02,4.000207
3,ID_-0.510_29.290_2022_03,4.000207
4,ID_-0.510_29.290_2022_04,4.007669
5,ID_-0.510_29.290_2022_05,4.065936
6,ID_-0.510_29.290_2022_06,4.115293
7,ID_-0.510_29.290_2022_07,4.142044
8,ID_-0.510_29.290_2022_08,4.155004
9,ID_-0.510_29.290_2022_09,4.153039
